# Seagate Exos X CSI driver functionality
Contact: Gregory Touretsky, gregory.touretsky@seagate.com

Seagate Exos X CSI driver supports Seagate Gallium controllers (including OEM versions).

Prerequisites: Exos X enclosure is configured with iSCSI. Pool[s] are created

## Review Kubernetes cluster

In [1]:
# Current CSI driver functionality should work with Kubernetes v.1.20 and higher
# Some features may not work with older versions of Kubernetes
kubectl get nodes

NAME          STATUS   ROLES                  AGE     VERSION
ssc-vm-3914   Ready    control-plane,master   7d15h   v1.21.0
ssc-vm-3915   Ready    <none>                 7d15h   v1.21.0


## Install CSI driver using Helm

In [2]:
# As a prerequisite, clone Seagate Exos X CSI driver from https://github.com/Seagate/seagate-exos-x-csi
# Container images will be pulled from GitHub Container Repository
helm install seagate-csi -n seagate --create-namespace \
    ../../seagate-exos-x-csi/helm/csi-charts \
    -f ../../seagate-exos-x-csi/helm/csi-charts/values.yaml

NAME: seagate-csi
LAST DEPLOYED: Thu Jul  8 10:07:45 2021
NAMESPACE: seagate
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
Thank you for using Seagate Exos X provisioner. It will be up and running shortly.
Run 'kubectl get pods' to verify that the new pods have a 'STATUS' of 'Running'.

In order to dynamically provide a persistant volume, create a storage class first.
Please refer to this example to do so: https://github.com/Seagate/seagate-exos-x-csi/blob/main/example/storage-class.yaml


In [3]:
# We recommend to install CSI driver in a dedicated namespace. Here it's running in a namespace "seagate"
# There is a single Controller (seagate-csi-controller) instance per cluster 
# and a single Node (seagate-csi-node) instance per worker node 
kubectl get pods -n seagate -o wide

NAME                                                    READY   STATUS    RESTARTS   AGE   IP               NODE          NOMINATED NODE   READINESS GATES
seagate-exos-x-csi-controller-server-66db6569fb-tf9sz   5/5     Running   0          55s   10.244.1.25      ssc-vm-3915   <none>           <none>
seagate-exos-x-csi-node-server-4xfnh                    3/3     Running   0          55s   10.230.241.7     ssc-vm-3915   <none>           <none>
seagate-exos-x-csi-node-server-kz78z                    3/3     Running   0          55s   10.230.242.215   ssc-vm-3914   <none>           <none>


## Create a secret for every managed enclosure

In [9]:
# Secret contains base64-encoded management IP and credentials for the enclosure
# For security reasons, this secret should be created in the same namespace where the driver is installed (seagate)
kubectl create -f iscsi/secret-exos-x.yaml

secret/seagate-enclosure001 created


## Create a Storage Class

In [4]:
# Some of these parameters (such as iqn and portals) will be deprecated in the future release
# Other parameters will be added
cat iscsi/storage-class.yaml

apiVersion: storage.k8s.io/v1
kind: StorageClass
provisioner: csi-exos-x.seagate.com
allowVolumeExpansion: true
metadata:
  name: seagate-storageclass-demo
parameters:
  csi.storage.k8s.io/provisioner-secret-name: seagate-enclosure001
  csi.storage.k8s.io/provisioner-secret-namespace: seagate
  csi.storage.k8s.io/controller-publish-secret-name: seagate-enclosure001
  csi.storage.k8s.io/controller-publish-secret-namespace: seagate
  csi.storage.k8s.io/controller-expand-secret-name: seagate-enclosure001
  csi.storage.k8s.io/controller-expand-secret-namespace: seagate
  fsType: ext4 # Desired filesystem
  iqn: iqn.2015-11.com.hpe:storage.msa2040.163639c929 # Gallium 10.235.212.197 Appliance IQN
  pool: A # Pool for volumes provisioning
  portals: 10.235.212.199,10.235.212.200 # Comma separated list of portal ips. Gallium 10.235.212.197 Ports A1, B1


In [5]:
kubectl create -f iscsi/storage-class.yaml

storageclass.storage.k8s.io/seagate-storageclass-demo created


In [6]:
kubectl get sc seagate-storageclass-demo

NAME                        PROVISIONER              RECLAIMPOLICY   VOLUMEBINDINGMODE   ALLOWVOLUMEEXPANSION   AGE
seagate-storageclass-demo   csi-exos-x.seagate.com   Delete          Immediate           true                   4s


## Validate lack of PVCs and pods in the demo space

In [7]:
# In this demo PVCs and pods will be created in a namespace "demo"
# At the beginning of the demo this namespace is empty
kubectl get pvc,volumesnapshot,pod -n demo

No resources found in demo namespace.


## Create a Persistent Volume Claim

In [16]:
cat iscsi/pvc.yaml

apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: seagate-pvc-demo
  namespace: demo
spec:
  accessModes:
    - ReadWriteOnce
  storageClassName: seagate-storageclass-demo
  resources:
    requests:
      storage: 1Gi


In [17]:
kubectl create -f iscsi/pvc.yaml

persistentvolumeclaim/seagate-pvc-demo created


In [18]:
kubectl get pvc seagate-pvc-demo -n demo

NAME               STATUS   VOLUME                                     CAPACITY   ACCESS MODES   STORAGECLASS                AGE
seagate-pvc-demo   Bound    pvc-fe11b209-b694-4e37-9c8d-5dc1b020ef34   1Gi        RWO            seagate-storageclass-demo   32s


## Check newly created persistent volume in the enclosure UI
![Volume Screenshot](pics/new_pv.png)

## Create a snapshot class

In [19]:
cat iscsi/snapshot-class.yaml

apiVersion: snapshot.storage.k8s.io/v1
kind: VolumeSnapshotClass
metadata:
  name: seagate-snapshotclass-demo
driver: systems.csi.seagate.io
deletionPolicy: Delete
parameters:
  csi.storage.k8s.io/snapshotter-secret-name: seagate-enclosure001
  csi.storage.k8s.io/snapshotter-secret-namespace: seagate



In [20]:
kubectl create -f iscsi/snapshot-class.yaml

volumesnapshotclass.snapshot.storage.k8s.io/seagate-snapshotclass-demo created


In [21]:
kubectl get volumesnapshotclass

NAME                         DRIVER                   DELETIONPOLICY   AGE
seagate-snapshotclass-demo   systems.csi.seagate.io   Delete           14s


## Create a snapshot

In [22]:
cat iscsi/snapshot.yaml

apiVersion: snapshot.storage.k8s.io/v1
kind: VolumeSnapshot
metadata:
  name: seagate-pvc-snapshot-demo
  namespace: demo
spec:
  volumeSnapshotClassName: seagate-snapshotclass-demo
  source:
    persistentVolumeClaimName: seagate-pvc-demo


In [23]:
kubectl create -f iscsi/snapshot.yaml

volumesnapshot.snapshot.storage.k8s.io/seagate-pvc-snapshot-demo created


In [24]:
kubectl get volumesnapshot -n demo

NAME                        READYTOUSE   SOURCEPVC          SOURCESNAPSHOTCONTENT   RESTORESIZE   SNAPSHOTCLASS                SNAPSHOTCONTENT                                    CREATIONTIME   AGE
seagate-pvc-snapshot-demo   false        seagate-pvc-demo                                         seagate-snapshotclass-demo   snapcontent-8ecbfda3-cf67-4bb1-a6da-f81555d1daa1                  45s


## Check newly created snapshot in the enclosure UI
![Snapshot Screenshot](pics/new_snapshot.png)

## Create a new PVC from the snapshot (restore)

In [26]:
cat iscsi/restoresnapshot.yaml

apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: seagate-snapshot-pvc-restore-demo
  namespace: demo
spec:
  dataSource:
    name: seagate-pvc-snapshot-demo
    kind: VolumeSnapshot
    apiGroup: "snapshot.storage.k8s.io"
  accessModes:
    - ReadWriteOnce
  storageClassName: seagate-storageclass-demo
  resources:
    requests:
      storage: 1Gi


## <font color='red'>Restore functionality is not implemented yet</font>

## Create a new PVC from the existing PVC (clone)

In [27]:
cat iscsi/clonepvc.yaml

apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: seagate-pvc-clone-demo
  namespace: demo
spec:
  dataSource:
    name: seagate-pvc-demo
    kind: PersistentVolumeClaim
  accessModes:
    - ReadWriteOnce
  storageClassName: seagate-storageclass-demo
  resources:
    requests:
      storage: 1Gi


In [28]:
kubectl create -f iscsi/clonepvc.yaml

persistentvolumeclaim/seagate-pvc-clone-demo created


In [29]:
kubectl get pvc seagate-pvc-clone-demo -n demo

NAME                     STATUS   VOLUME                                     CAPACITY   ACCESS MODES   STORAGECLASS                AGE
seagate-pvc-clone-demo   Bound    pvc-8383cd38-21b8-4b7e-a859-bc57854473d0   1Gi        RWO            seagate-storageclass-demo   2m55s


## Create a pod with the PVC attached

In [31]:
cat iscsi/app.yaml

apiVersion: v1
kind: Pod
metadata:
  name: seagate-pod-pvc-demo
  namespace: demo
spec:
  containers:
  - image: busybox
    name: theapp
    volumeMounts:
    - mountPath: /vol
      name: seagate-csi-volume
    command: [ "sleep", "1000" ]
  volumes:
  - name: seagate-csi-volume
    persistentVolumeClaim:
      claimName: seagate-pvc-demo
  nodeSelector:
    kubernetes.io/hostname: ssc-vm-3914



In [32]:
kubectl create -f iscsi/app.yaml

pod/seagate-pod-pvc-demo created


In [33]:
kubectl get pod -n demo

NAME                   READY   STATUS    RESTARTS   AGE
seagate-pod-pvc-demo   1/1     Running   0          78s


In [34]:
kubectl exec -it seagate-pod-pvc-demo -n demo -- df -h /vol

Filesystem                Size      Used Available Use% Mounted on
/dev/mapper/3600c0ff0000127730dcadc6001000000
                        975.9M      2.5M    906.2M   0% /vol


## Resize the PV

```
kubectl edit pvc seagate-pvc-demo -n demo
```
Replace 1Gi with new size (2Gi) in the spec:
```
spec:
  accessModes:
  - ReadWriteOnce
  resources:
    requests:
      storage: 1Gi
  storageClassName: seagate-storageclass-demo
  volumeMode: Filesystem
```

In [4]:
# Command above results in volume resize
# The pod must be restarted to complete file system resize 
kubectl delete -f iscsi/app.yaml

pod "seagate-pod-pvc-demo" deleted


In [1]:
kubectl create -f iscsi/app.yaml

pod/seagate-pod-pvc-demo created


In [2]:
# Notice PVC size changed to 2Gi
kubectl get pvc,pod -n demo

NAME                                     STATUS   VOLUME                                     CAPACITY   ACCESS MODES   STORAGECLASS                AGE
persistentvolumeclaim/seagate-pvc-demo   Bound    pvc-c543db9e-ef03-4566-83b2-f785d9db5cf0   2Gi        RWO            seagate-storageclass-demo   6m19s

NAME                       READY   STATUS    RESTARTS   AGE
pod/seagate-pod-pvc-demo   1/1     Running   0          35s


In [3]:
# And the filesystem is 1.9G now
kubectl exec -it seagate-pod-pvc-demo -n demo -- df -h /vol

Filesystem                Size      Used Available Use% Mounted on
/dev/mapper/3600c0ff000012773fb9bdd6001000000
                          1.9G      3.0M      1.8G   0% /vol


## Metrics
Exos X CSI driver exports a variety of metrics which can be used to track usage and performance of the driver.
By default, Controller exposes /metrics on port tcp/9842.
Also, every Node exposes /healthz endpoint on port tcp/9808 which returns "ok"

We'll publish a reference Prometheus query and Grafana dashboard in the future.
For now, here is the query directly to the exporter 

In [2]:
# Define IP of the Controller pod
CTRL_IP=`kubectl get pod -n seagate -o wide  | grep seagate-exos-x-csi-controller | awk '{print $6}'`
# use curl to query the pod
curl -s http://$CTRL_IP:9842/metrics


# HELP exosx_api_appliance_api_call How many API calls have been executed
# TYPE exosx_api_appliance_api_call counter
exosx_api_appliance_api_call{endpoint="/create/snapshots/volumes/%q/%q",success="false"} 40
exosx_api_appliance_api_call{endpoint="/create/snapshots/volumes/%q/%q",success="true"} 1
exosx_api_appliance_api_call{endpoint="/create/volume/pool/\"%s\"/size/%s/tier-affinity/no-affinity/\"%s\"",success="true"} 1
exosx_api_appliance_api_call{endpoint="/delete/volumes/\"%s\"",success="true"} 1
exosx_api_appliance_api_call{endpoint="/login/%s",success="true"} 1
exosx_api_appliance_api_call{endpoint="/show/snapshots/%q",success="false"} 41
exosx_api_appliance_api_call{endpoint="/show/volumes/\"%s\"",success="false"} 1
# HELP exosx_api_appliance_api_call_duration The total duration of API calls
# TYPE exosx_api_appliance_api_call_duration counter
exosx_api_appliance_api_call_duration{endpoint="/create/snapshots/volumes/%q/%q"} 6.616106189
exosx_api_appliance_api_call_duration{endp

seagate_csi_rpc_call{endpoint="/csi.v1.Identity/Probe",success="true"} 4
# HELP seagate_csi_rpc_call_duration The total duration of CSI RPC calls
# TYPE seagate_csi_rpc_call_duration counter
seagate_csi_rpc_call_duration{endpoint="/csi.v1.Controller/ControllerGetCapabilities"} 0.000184456
seagate_csi_rpc_call_duration{endpoint="/csi.v1.Controller/CreateSnapshot"} 10.206516281999999
seagate_csi_rpc_call_duration{endpoint="/csi.v1.Controller/CreateVolume"} 1.103964113
seagate_csi_rpc_call_duration{endpoint="/csi.v1.Controller/DeleteVolume"} 0.535178527
seagate_csi_rpc_call_duration{endpoint="/csi.v1.Identity/GetPluginCapabilities"} 7.6364e-05
seagate_csi_rpc_call_duration{endpoint="/csi.v1.Identity/GetPluginInfo"} 0.0030613609999999994
seagate_csi_rpc_call_duration{endpoint="/csi.v1.Identity/Probe"} 0.00022928499999999997


## Clean everything

In [35]:
kubectl delete -f iscsi/app.yaml

pod "seagate-pod-pvc-demo" deleted


In [36]:
kubectl delete -f iscsi/clonepvc.yaml

persistentvolumeclaim "seagate-pvc-clone-demo" deleted


In [37]:
kubectl delete -f iscsi/restoresnapshot.yaml

persistentvolumeclaim "seagate-snapshot-pvc-restore-demo" deleted


In [38]:
# There is a bug in the current version, snapshot is not actually deleted on the enclosure
kubectl delete -f iscsi/snapshot.yaml

volumesnapshot.snapshot.storage.k8s.io "seagate-pvc-snapshot-demo" deleted


In [39]:
kubectl delete -f iscsi/pvc.yaml

persistentvolumeclaim "seagate-pvc-demo" deleted


In [40]:
kubectl delete -f iscsi/snapshot-class.yaml

volumesnapshotclass.snapshot.storage.k8s.io "seagate-snapshotclass-demo" deleted


In [41]:
kubectl delete -f iscsi/storage-class.yaml

storageclass.storage.k8s.io "seagate-storageclass-demo" deleted


In [42]:
helm uninstall seagate-csi -n seagate

release "seagate-csi" uninstalled


## Future functionality
* Restore from snapshot
* OpenShift operator
----
* Additional parameters in StorageClass definition
* Managing multiple enclosures / pools via single StorageClass
* Topology 
* Capacity
* Import a volume
* Raw block device
* iSCSI CHAP
* SAS and NVMe connectivity support
* Support for Windows